In [20]:
# Standard PyTorch imports
import torch
import pandas as pd
from torch import nn
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 5),
            nn.ReLU(),
            nn.Linear(5, 3),
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [22]:
model = NeuralNetwork().to(device)

In [23]:
df = pd.read_csv("ModelGrader/modelgrader_submission_processed.csv",index_col="problem_id")
df['difficulty'] = df['difficulty'] - 1
df

,avg_first_passed_total_attempts,avg_first_passed_time_used,difficulty
problem_id,,,
100,32,4909.323282,0
101,21,339.843212,0
102,27,7609.605477,1
103,11,1504.160602,0
104,10,2221.719998,0
...,...,...,...
96,20,128.579397,0
97,37,685.743135,0
98,17,33.388133,0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(df[["avg_first_passed_total_attempts","avg_first_passed_time_used"]], df["difficulty"], test_size=0.2, random_state=42)

X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.long).to(device)

X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.long).to(device)

y_train

tensor([1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1],
       device='cuda:0')

In [25]:
# Create a loss function
# loss_fn = nn.BCELoss() # BCELoss = no sigmoid built-in
loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

# Create an optimizer
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

In [26]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [27]:
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([93, 2]), torch.Size([93]), torch.Size([24, 2]), torch.Size([24]))

In [31]:
torch.manual_seed(42)

# Set the number of epochs
epochs = 1

# Put data to target device
# X_train, y_train = X_train.to(device), y_train.to(device)
# X_test, y_test = X_test.to(device), y_test.to(device)

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model.train()


    y_logits = model(X_train)
    pred_prob = torch.sigmoid(y_logits)
    y_pred = pred_prob.argmax(dim=1)
    # print(y_pred, y_train)

    loss = loss_fn(y_pred.float(), y_train.float())
    loss = Variable(loss, requires_grad = True)

    test_acc = accuracy_fn(y_true=y_train,y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model(X_test).squeeze() 
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        y_test = test_logits.squeeze()
        print(test_logits, y_test)
        test_loss = loss_fn(test_logits,y_test)
        test_acc = accuracy_fn(y_true=y_test,y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {test_acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

tensor([[ 5.1852e-01, -1.6317e+00, -1.1079e+00],
        [ 2.4365e+02, -1.3058e+02, -4.9794e+02],
        [ 8.8406e+02, -4.6722e+02, -1.8056e+03],
        [ 1.3479e+01, -9.7610e+00, -2.8083e+01],
        [ 1.0421e+02, -5.6192e+01, -2.1253e+02],
        [ 1.8923e+02, -1.0250e+02, -3.8716e+02],
        [ 1.5220e+01, -1.0678e+01, -3.1639e+01],
        [ 3.6133e+00, -3.2201e+00, -7.0655e+00],
        [ 8.3220e+00, -5.8918e+00, -1.6806e+01],
        [ 9.3139e+01, -5.0554e+01, -1.9004e+02],
        [ 1.2562e+01, -7.9324e+00, -2.5343e+01],
        [ 3.1055e+01, -1.7093e+01, -6.2742e+01],
        [ 3.6367e+01, -2.1044e+01, -7.4335e+01],
        [ 5.8995e+00, -4.2317e+00, -1.1611e+01],
        [ 7.5407e-01, -1.1377e+00, -8.5400e-01],
        [ 5.3758e+02, -2.8958e+02, -1.1010e+03],
        [ 1.4672e+03, -7.7679e+02, -2.9982e+03],
        [ 1.3926e+01, -1.1535e+01, -2.9988e+01],
        [ 2.8181e+02, -1.5798e+02, -5.8059e+02],
        [ 2.8995e+01, -1.6970e+01, -5.9155e+01],
        [ 1.6243e+02